In [ ]:
from glob import glob
import cv2
import os
import tensorflow as tf
import pandas as pd
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
fileList = glob("./shuffle_data_gzip/*.csv.gz")     

In [5]:
def draw_cv2(raw_strokes,BASE_SIZE=256, size=256, lw=6):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for stroke in raw_strokes:
        for i in range(len(stroke[0]) - 1):
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), 255, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size)) 
    else:
        return img

In [6]:
tfrecord_file = "./tfrecord_data/train.tfrecords"

In [7]:
with tf.io.TFRecordWriter(tfrecord_file) as writer:
    for filename in fileList[:1]:
        df = pd.read_csv(filename)
        df['drawing'] = df['drawing'].apply(json.loads)
        for row in range(df.shape[0]):
            drawing = df.loc[row,'drawing']
            img = draw_cv2(drawing,BASE_SIZE=128, size=128, lw=6)
            img = img.tostring()
            label = df.loc[row,'y']
            # 建立 tf.train.Feature 字典
            feature = {                             
                    'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img])),  # 图片是一个 Bytes 对象
                    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))   # 标签是一个 Int 对象
                }
            example = tf.train.Example(features=tf.train.Features(feature=feature)) # 通过字典建立 Example
            writer.write(example.SerializeToString())   # 将Example序列化并写入 TFRecord 文件

In [13]:
# 读取 TFRecord 文件
raw_dataset = tf.data.TFRecordDataset(tfrecord_file)    
# 定义Feature结构，告诉解码器每个Feature的类型是什么
feature_description = { 
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}

# 将 TFRecord 文件中的每一个序列化的 tf.train.Example 解码
def _parse_example(example_string): 
    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    image = tf.io.decode_raw(feature_dict['image'], tf.uint8)    # 解码JPEG图片
    image = tf.reshape(image, [128,128,1])
    image = tf.dtypes.cast(image,tf.float32)
    image = image / 255.0
    label = tf.one_hot(feature_dict['label'],depth=340)
    return image, label

dataset = raw_dataset.map(_parse_example)

In [14]:
train_ds = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE).shuffle(buffer_size=tf.data.experimental.AUTOTUNE).batch(1024)

In [15]:

class MobileNetModel(tf.keras.models.Model):
    def __init__(self, size, n_labels, **kwargs):
        super(MobileNetModel, self).__init__(**kwargs)
        self.base_model = tf.keras.applications.MobileNet(input_shape=(size, size, 1), include_top=False, weights=None, classes=n_labels)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(1024, activation='relu')
        self.outputs =  tf.keras.layers.Dense(n_labels, activation='softmax')

        
    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.flatten(x)
        x = self.dense(x)
        output_ = self.outputs(x)
        return output_


In [ ]:
#batchnorm

In [16]:
model = MobileNetModel(size=128,n_labels=340)

loss_object = tf.keras.losses.CategoricalCrossentropy()


learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_top3_accuracy = tf.keras.metrics.TopKCategoricalAccuracy(k=3,name='train_top_3_categorical_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_top3_accuracy = tf.keras.metrics.TopKCategoricalAccuracy(k=3,name='test_top_3_categorical_accuracy')


# @tf.function
def train_one_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
     
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    train_top3_accuracy(labels, predictions)

def val_one_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    test_top3_accuracy(labels, predictions)

In [ ]:
EPOCHS=10
for epoch in range(EPOCHS):
    # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    train_top3_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    test_top3_accuracy.reset_states()

    for step,(images, labels) in enumerate(train_ds):
        train_one_step(images, labels)
        
        if step % 200 == 0:
            print("step:{0}; Samples:{1}; Train Loss:{2}; Train Accuracy:{3},Train Top3 Accuracy:{4}".format(step, (step + 1) * 1024, 
                                                                                                             train_loss.result(), 
                                                                                                             train_accuracy.result()*100, 
                                                                                                             train_top3_accuracy.result()*100))




    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          train_top3_accuracy()*100,
                          test_loss.result(),
                          test_accuracy.result() * 100,
                          test_top3_accuracy()*100
                         ))

step:0; Samples:1024; Train Loss:5.828945636749268; Train Accuracy:0.0078125
step:200; Samples:205824; Train Loss:5.809243679046631; Train Accuracy:0.007520988583564758
step:400; Samples:410624; Train Loss:5.806193828582764; Train Accuracy:0.007296212483197451
step:600; Samples:615424; Train Loss:5.805119037628174; Train Accuracy:0.006967553868889809
step:800; Samples:820224; Train Loss:5.804376125335693; Train Accuracy:0.006993211805820465
step:1000; Samples:1025024; Train Loss:5.804078578948975; Train Accuracy:0.006914960220456123
step:1200; Samples:1229824; Train Loss:5.803527355194092; Train Accuracy:0.006833497900515795
step:1400; Samples:1434624; Train Loss:5.803006172180176; Train Accuracy:0.006864516530185938
step:1600; Samples:1639424; Train Loss:5.8026323318481445; Train Accuracy:0.006941462401300669
step:1800; Samples:1844224; Train Loss:5.802396774291992; Train Accuracy:0.00689720967784524
Epoch 1, Loss: 5.802236557006836, Accuracy: 0.6857188940048218, Test Loss: 0.0, Test 